# making of corpus

In [3]:
import pandas as pd
import duckdb
conn = duckdb.connect('/srv/data/greek/grela.duckdb', read_only=True)

In [5]:
query = '''
SELECT t.*, w.*, s.*
FROM tokens t
JOIN works w ON t.grela_id = w.grela_id
JOIN sentences s ON t.sentence_id = s.sentence_id
WHERE w.lagt_provenience IN ('christian', 'pagan')
  AND t.pos IN ('n', 'v', 'a')
  AND (
    (w.not_before > 0 AND w.not_before < 600)
    OR (w.not_after > 0 AND w.not_after < 600)
  )
'''
df = conn.execute(query).fetchdf()
df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,sentence_id,grela_id,token_text,lemma,pos,char_start,char_end,token_id,grela_source,grela_id_1,...,place_publication,place_geonames,author_viaf,title_viaf,date_random,token_count,sentence_id_1,grela_id_2,position,text
0,lagt_tlg0612.tlg001_3920,lagt_tlg0612.tlg001,φρονῶ,φρονέω,v,17,22,145334526,lagt,lagt_tlg0612.tlg001,...,None,None,NaN,NaN,89.0,209358,lagt_tlg0612.tlg001_3920,lagt_tlg0612.tlg001,3920,ἐγὼ μὲν οὖν οὕτω φρονῶ καὶ δέχομαι πρᾴως τὴν π...
1,lagt_tlg0612.tlg001_3920,lagt_tlg0612.tlg001,δέχομαι,δέχομαι,v,27,34,145334528,lagt,lagt_tlg0612.tlg001,...,None,None,NaN,NaN,89.0,209358,lagt_tlg0612.tlg001_3920,lagt_tlg0612.tlg001,3920,ἐγὼ μὲν οὖν οὕτω φρονῶ καὶ δέχομαι πρᾴως τὴν π...
2,lagt_tlg0612.tlg001_3920,lagt_tlg0612.tlg001,πεπρωμένην,πόρω,v,45,55,145334531,lagt,lagt_tlg0612.tlg001,...,None,None,NaN,NaN,89.0,209358,lagt_tlg0612.tlg001_3920,lagt_tlg0612.tlg001,3920,ἐγὼ μὲν οὖν οὕτω φρονῶ καὶ δέχομαι πρᾴως τὴν π...
3,lagt_tlg0612.tlg001_3920,lagt_tlg0612.tlg001,καιρῷ,καιρός,n,70,75,145334536,lagt,lagt_tlg0612.tlg001,...,None,None,NaN,NaN,89.0,209358,lagt_tlg0612.tlg001_3920,lagt_tlg0612.tlg001,3920,ἐγὼ μὲν οὖν οὕτω φρονῶ καὶ δέχομαι πρᾴως τὴν π...
4,lagt_tlg0612.tlg001_3920,lagt_tlg0612.tlg001,λέγων,λέγω,v,82,87,145334538,lagt,lagt_tlg0612.tlg001,...,None,None,NaN,NaN,89.0,209358,lagt_tlg0612.tlg001_3920,lagt_tlg0612.tlg001,3920,ἐγὼ μὲν οὖν οὕτω φρονῶ καὶ δέχομαι πρᾴως τὴν π...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10043230,lagt_tlg0082.tlg004_3220,lagt_tlg0082.tlg004,παρακλίναντα,παρακλίνω,v,68,80,165781515,lagt,lagt_tlg0082.tlg004,...,None,None,NaN,NaN,138.0,71099,lagt_tlg0082.tlg004_3220,lagt_tlg0082.tlg004,3220,"οὕτως ἔχει καὶ τὸ αὐτόδιον δ’ ἄρα μιν, ἀντὶ το..."
10043231,lagt_tlg0082.tlg004_3221,lagt_tlg0082.tlg004,παράκειται,παράκειμαι,v,0,10,165781517,lagt,lagt_tlg0082.tlg004,...,None,None,NaN,NaN,138.0,71099,lagt_tlg0082.tlg004_3221,lagt_tlg0082.tlg004,3221,παράκειται οὖν καὶ τῷ ἐξαίφνης ἄφνω παρεγένετο...
10043232,lagt_tlg0082.tlg004_3221,lagt_tlg0082.tlg004,παρεγένετο,παραγίγνομαι,v,36,46,165781523,lagt,lagt_tlg0082.tlg004,...,None,None,NaN,NaN,138.0,71099,lagt_tlg0082.tlg004_3221,lagt_tlg0082.tlg004,3221,παράκειται οὖν καὶ τῷ ἐξαίφνης ἄφνω παρεγένετο...
10043233,lagt_tlg0082.tlg004_3221,lagt_tlg0082.tlg004,ἴσῳ,ἴσος,a,58,61,165781527,lagt,lagt_tlg0082.tlg004,...,None,None,NaN,NaN,138.0,71099,lagt_tlg0082.tlg004_3221,lagt_tlg0082.tlg004,3221,παράκειται οὖν καὶ τῷ ἐξαίφνης ἄφνω παρεγένετο...


0-300
300-600

In [ ]:
import json

grouped = (
    df.groupby('sentence_id').agg(
        lamma_sentence=pd.NamedAgg(column='lemma', aggfunc=lambda x: ' '.join(x)),
        token_ids=pd.NamedAgg(column='token_id', aggfunc=list),
        title=pd.NamedAgg(column='title', aggfunc='first'),
        author=pd.NamedAgg(column='author', aggfunc='first'),
        lagt_provenience=pd.NamedAgg(column='lagt_provenience', aggfunc='first'),
        not_before=pd.NamedAgg(column='not_before', aggfunc='first'),
        not_after=pd.NamedAgg(column='not_after', aggfunc='first'),
        sentence_text=pd.NamedAgg(column='text', aggfunc='first'),
        lemma_list=pd.NamedAgg(column='lemma', aggfunc=list),
        pos_list=pd.NamedAgg(column='pos', aggfunc=list),
        char_start_list=pd.NamedAgg(column='char_start', aggfunc=list),
        char_end_list=pd.NamedAgg(column='char_end', aggfunc=list),
    ).reset_index()
)

grouped['tokens'] = [
    json.dumps([
        {
            'lemma': lemma,
            'pos': pos,
            'char_start': char_start,
            'char_end': char_end
        }
        for lemma, pos, char_start, char_end in zip(row['lemma_list'], row['pos_list'], row['char_start_list'], row['char_end_list'])
    ], ensure_ascii=False)
    for _, row in grouped.iterrows()
]

grouped = grouped.drop(columns=['lemma_list', 'pos_list', 'char_start_list', 'char_end_list'])

grouped.head()

,sentence_id,lamma_sentence,token_ids,title,author,lagt_provenience,not_before,not_after,sentence_text,tokens
0,lagt_ogl0001.ogl001_4,πινυτός ἀντιγράφω,"[157411875, 157411876]",De Epistola Pinyti ad Dionysium,Pinytus,christian,101.0,200.0,"ὁ Πινυτός ἀντιγράφων,","[{""lemma"": ""πινυτός"", ""pos"": ""n"", ""char_start""..."
1,lagt_ogl0001.ogl001_5,θαυμάζω ἀποδέχω διονύσιος,"[157411878, 157411881, 157411883]",De Epistola Pinyti ad Dionysium,Pinytus,christian,101.0,200.0,θαυμάζει μέν καί ἀποδέχεται τόν Διονύσιον·,"[{""lemma"": ""θαυμάζω"", ""pos"": ""v"", ""char_start""..."
2,lagt_ogl0001.ogl001_6,ἀντιπαρακαλέω στεῤῥοτέρας μεταδίδωμι τροφή,"[157411885, 157411887, 157411890, 157411891]",De Epistola Pinyti ad Dionysium,Pinytus,christian,101.0,200.0,ἀντιπαρακαλεῖ δέ στεῤῥοτέρας ἤδη ποτέ μεταδιδό...,"[{""lemma"": ""ἀντιπαρακαλέω"", ""pos"": ""v"", ""char_..."
3,lagt_ogl0001.ogl001_7,τελειοτέρω γράμμα λαός ὑποθρέψαντα διατέλος γα...,"[157411893, 157411894, 157411899, 157411900, 1...",De Epistola Pinyti ad Dionysium,Pinytus,christian,101.0,200.0,τελειοτέροις γράμμασιν εἰσαῦθις τόν παῤ αὐτῷ λ...,"[{""lemma"": ""τελειοτέρω"", ""pos"": ""a"", ""char_sta..."
4,lagt_ogl0001.ogl001_8,ἐπιστολή πινυτός πίστις ὀρθοδοξία φροντίς ὑπήκ...,"[157411918, 157411922, 157411925, 157411926, 1...",De Epistola Pinyti ad Dionysium,Pinytus,christian,101.0,200.0,δι᾿ ἧς ἐπιστολῆς καί ἡ τοῦ Πινυτοῦ περί τήν πί...,"[{""lemma"": ""ἐπιστολή"", ""pos"": ""n"", ""char_start..."


In [12]:
grouped.to_pickle('../data/large-data/grouped_df.pkl')

In [5]:
# Split by 'lagt_provenience'
grouped_christian = grouped[grouped['lagt_provenience'] == 'christian']
grouped_pagan = grouped[grouped['lagt_provenience'] == 'pagan']

christian_0_300 = grouped_christian[
    (grouped_christian['not_before'] >= 0) & (grouped_christian['not_after'] <= 300)
]
christian_300_600 = grouped_christian[
    (grouped_christian['not_before'] >= 300) & (grouped_christian['not_after'] <= 600)
]

pagan_0_300 = grouped_pagan[
    (grouped_pagan['not_before'] >= 0) & (grouped_pagan['not_after'] <= 300)
]
pagan_300_600 = grouped_pagan[
    (grouped_pagan['not_before'] >= 300) & (grouped_pagan['not_after'] <= 600)
]

# fasttext

In [ ]:
from nltk.probability import FreqDist

fdist1 = FreqDist(word for sent in christian_0_300['lamma_sentence'] for word in sent.split())
fdist2 = FreqDist(word for sent in christian_300_600['lamma_sentence'] for word in sent.split())
fdist3 = FreqDist(word for sent in pagan_0_300['lamma_sentence'] for word in sent.split())
fdist4 = FreqDist(word for sent in pagan_300_600['lamma_sentence'] for word in sent.split())

# Find words that appear >=10 times in all subcorpora
common_words = set(
    word for word in fdist1 if fdist1[word] >= 10
) & set(
    word for word in fdist2 if fdist2[word] >= 10
) & set(
    word for word in fdist3 if fdist3[word] >= 10
) & set(
    word for word in fdist4 if fdist4[word] >= 10
)

# Filter each subcorpus to only keep those words
words1 = {word: fdist1[word] for word in common_words}
words2 = {word: fdist2[word] for word in common_words}
words3 = {word: fdist3[word] for word in common_words}
words4 = {word: fdist4[word] for word in common_words}

In [ ]:
from gensim.models import FastText
from gensim.models.callbacks import CallbackAny2Vec
from gensim.utils import RULE_KEEP, RULE_DISCARD, RULE_DEFAULT

class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
    def on_epoch_begin(self, model):
        print(f"Epoch #{self.epoch} start")
    def on_epoch_end(self, model):
        print(f"Epoch #{self.epoch} end")
        self.epoch += 1

def train_fasttext_on_subcorpus(sentences, model_path):
    #sentences = [s.split() for s in sentences]
    model = FastText(
        vector_size=100,
        window=5, #vojta used 10
        min_count=1,
        sg=1,
        workers=16,
    )
    model.build_vocab_from_freq(word_freq=sentences)
    model.train(
        corpus_iterable=sentences,
        total_examples=len(sentences),
        epochs=20,
        callbacks=[EpochLogger()]
    )
    model.save(model_path)
    return model


In [9]:
# Example usage for each subcorpus:
christian_0_300_model = train_fasttext_on_subcorpus(
    words1, "../data/large-data/fasttext_christian_0_300.model"
)
christian_300_600_model = train_fasttext_on_subcorpus(
    words2, "../data/large-data/fasttext_christian_300_600.model"
)
pagan_0_300_model = train_fasttext_on_subcorpus(
    words3, "../data/large-data/fasttext_pagan_0_300.model"
)
pagan_300_600_model = train_fasttext_on_subcorpus(
    words4, "../data/large-data/fasttext_pagan_300_600.model"
)

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end
Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoc

In [1]:
from gensim.models import FastText

# Load every FastText model for each subcorpus into a variable
christian_0_300_model = FastText.load("../data/large-data/fasttext_christian_0_300.model")
christian_300_600_model = FastText.load("../data/large-data/fasttext_christian_300_600.model")
pagan_0_300_model = FastText.load("../data/large-data/fasttext_pagan_0_300.model")
pagan_300_600_model = FastText.load("../data/large-data/fasttext_pagan_300_600.model")

In [10]:
%%time
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

models = [
    (christian_0_300_model, "christian_0_300", 0),
    (christian_300_600_model, "christian_300_600", 1),
    (pagan_0_300_model, "pagan_0_300", 2),
    (pagan_300_600_model, "pagan_300_600", 3),
]

dfs = []
for model, label, z in models:
    vocab = list(model.wv.index_to_key)
    vecs = [model.wv[w] for w in vocab]
    tsne = TSNE(n_components=2, random_state=42, verbose=1)
    tsne_result = tsne.fit_transform(np.array(vecs))
    df = pd.DataFrame({
        "word": vocab,
        "x": tsne_result[:, 0],
        "y": tsne_result[:, 1],
        "z": z,
        "subcorpus": label
    })
    dfs.append(df)

df_plot = pd.concat(dfs, ignore_index=True)
df_plot.to_pickle("../data/large-data/df_plot.pkl")

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3561 samples in 0.001s...
[t-SNE] Computed neighbors for 3561 samples in 0.079s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3561
[t-SNE] Computed conditional probabilities for sample 2000 / 3561
[t-SNE] Computed conditional probabilities for sample 3000 / 3561
[t-SNE] Computed conditional probabilities for sample 3561 / 3561
[t-SNE] Mean sigma: 0.003331
[t-SNE] KL divergence after 250 iterations with early exaggeration: 81.659798
[t-SNE] KL divergence after 1000 iterations: 3.194238
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3561 samples in 0.000s...
[t-SNE] Computed neighbors for 3561 samples in 0.035s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3561
[t-SNE] Computed conditional probabilities for sample 2000 / 3561
[t-SNE] Computed conditional probabilities for sample 3000 / 3561
[t-SNE] Computed conditional probabilities for sample 3561 / 3561
[t-SNE] Mean sigma: 0.003326


In [3]:
df_plot = pd.read_pickle("../data/large-data/df_plot.pkl")

In [11]:
%%time
# Precompute nearest neighbors for all words in all subcorpora and save to pickle
from collections import defaultdict

neighbors_dict = defaultdict(dict)
model_map = {
    "christian_0_300": christian_0_300_model,
    "christian_300_600": christian_300_600_model,
    "pagan_0_300": pagan_0_300_model,
    "pagan_300_600": pagan_300_600_model,
}
N_NEIGHBORS = 30

for subcorpus, model in model_map.items():
    for word in model.wv.index_to_key:
        try:
            neighbors = model.wv.most_similar(word, topn=N_NEIGHBORS)  # returns list of (word, similarity)
        except KeyError:
            neighbors = []
        neighbors_dict[subcorpus][word] = neighbors

import pickle
with open("../data/large-data/word_neighbors.pkl", "wb") as f:
    pickle.dump(dict(neighbors_dict), f)

CPU times: user 12.3 s, sys: 41.1 s, total: 53.4 s
Wall time: 1.68 s


# TF-IDF

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

grouped = pd.read_pickle('../data/large-data/grouped_df.pkl')

In [3]:
# Split by 'lagt_provenience'
grouped_christian = grouped[grouped['lagt_provenience'] == 'christian']
grouped_pagan = grouped[grouped['lagt_provenience'] == 'pagan']

christian_0_300 = grouped_christian[
    (grouped_christian['not_before'] >= 0) & (grouped_christian['not_after'] <= 300)
]
christian_300_600 = grouped_christian[
    (grouped_christian['not_before'] >= 300) & (grouped_christian['not_after'] <= 600)
]

pagan_0_300 = grouped_pagan[
    (grouped_pagan['not_before'] >= 0) & (grouped_pagan['not_after'] <= 300)
]
pagan_300_600 = grouped_pagan[
    (grouped_pagan['not_before'] >= 300) & (grouped_pagan['not_after'] <= 600)
]

In [4]:
from nltk.probability import FreqDist

# Find most frequent words (lowercased)
fdist1 = FreqDist(word.lower() for sent in christian_0_300['lamma_sentence'] for word in sent.split())
fdist2 = FreqDist(word.lower() for sent in christian_300_600['lamma_sentence'] for word in sent.split())
fdist3 = FreqDist(word.lower() for sent in pagan_0_300['lamma_sentence'] for word in sent.split())
fdist4 = FreqDist(word.lower() for sent in pagan_300_600['lamma_sentence'] for word in sent.split())

# Find words that appear >=10 times in all subcorpora
vocab = set(
    word for word in fdist1 if fdist1[word] >= 10
) & set(
    word for word in fdist2 if fdist2[word] >= 10
) & set(
    word for word in fdist3 if fdist3[word] >= 10
) & set(
    word for word in fdist4 if fdist4[word] >= 10
)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

def compute_tfidf(sentences):
    vectorizer = TfidfVectorizer(
        #min_df=10,
        #max_features=1000,
        vocabulary=vocab, #use common vocab
    )
    tfidf_matrix = vectorizer.fit_transform(sentences)
    feature_names = vectorizer.get_feature_names_out()
    return [tfidf_matrix, feature_names]

In [6]:
tfidf_christian_0_300 = compute_tfidf(christian_0_300['lamma_sentence'])
tfidf_christian_300_600 = compute_tfidf(christian_300_600['lamma_sentence'])
tfidf_pagan_0_300 = compute_tfidf(pagan_0_300['lamma_sentence'])
tfidf_pagan_300_600 = compute_tfidf(pagan_300_600['lamma_sentence'])

In [1]:
import numpy as np

def compute_weighted_cooccurrence(tfidf_data):
    """
    Given [tfidf_matrix, feature_names] as returned by compute_tfidf,
    returns a DataFrame of TF-IDF weighted term-term association.
    """
    tfidf_matrix, feature_names = tfidf_data
    weighted_cooccurrence = (tfidf_matrix.T @ tfidf_matrix)
    # Remove self-association (diagonal)
    np.fill_diagonal(weighted_cooccurrence.toarray(), 0)
    return pd.DataFrame(
        weighted_cooccurrence.toarray(),
        index=feature_names,
        columns=feature_names
    )

In [9]:
import pickle

# Compute co-occurrence matrices for all subcorpora
cooc_christian_0_300 = compute_weighted_cooccurrence(tfidf_christian_0_300)
cooc_christian_300_600 = compute_weighted_cooccurrence(tfidf_christian_300_600)
cooc_pagan_0_300 = compute_weighted_cooccurrence(tfidf_pagan_0_300)
cooc_pagan_300_600 = compute_weighted_cooccurrence(tfidf_pagan_300_600)

tfidf_cooc = {
    "christian_0_300": cooc_christian_0_300,
    "christian_300_600": cooc_christian_300_600,
    "pagan_0_300": cooc_pagan_0_300,
    "pagan_300_600": cooc_pagan_300_600,
}

with open("../data/large-data/tfidf_cooc.pkl", "wb") as f:
    pickle.dump(tfidf_cooc, f)

In [8]:
def strongest_cooccurrences(cooc_df, target_word, top_n=20):
    """
    Returns the top_n strongest co-occurring words with target_word from the co-occurrence DataFrame.
    """
    if target_word not in cooc_df.columns:
        print(f"'{target_word}' not found in vocabulary.")
        return None
    # Get co-occurrence scores for the target word, sort descending, exclude itself
    scores = cooc_df[target_word].drop(target_word).sort_values(ascending=False)
    return scores.head(top_n)

strongest_cooccurrences(coocdf, "ἐχθρός", top_n=20)

ἀγαπάω      9.943617
γίγνομαι    8.994032
φίλος       8.458694
θάνατος     8.009502
χείρ        7.731181
εἰμί        7.465912
θεός        7.230949
ἔσχατος     6.908857
λέγω        6.866909
πούς        6.706503
τίθημι      5.865670
μισέω       4.522370
ποιέω       4.178767
διώκω       4.036005
οὗτος       3.825989
χριστός     3.681934
κύριος      3.546914
γῆ          3.438901
πατήρ       3.412090
υἱός        3.386424
Name: ἐχθρός, dtype: float64

# PMI

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

grouped = pd.read_pickle('../data/large-data/grouped_df.pkl')

In [2]:
# Split by 'lagt_provenience'
grouped_christian = grouped[grouped['lagt_provenience'] == 'christian']
grouped_pagan = grouped[grouped['lagt_provenience'] == 'pagan']

christian_0_300 = grouped_christian[
    (grouped_christian['not_before'] >= 0) & (grouped_christian['not_after'] <= 300)
]
christian_300_600 = grouped_christian[
    (grouped_christian['not_before'] >= 300) & (grouped_christian['not_after'] <= 600)
]

pagan_0_300 = grouped_pagan[
    (grouped_pagan['not_before'] >= 0) & (grouped_pagan['not_after'] <= 300)
]
pagan_300_600 = grouped_pagan[
    (grouped_pagan['not_before'] >= 300) & (grouped_pagan['not_after'] <= 600)
]

In [3]:
from nltk.probability import FreqDist

# Find most frequent words (lowercased)
fdist1 = FreqDist(word.lower() for sent in christian_0_300['lamma_sentence'] for word in sent.split())
fdist2 = FreqDist(word.lower() for sent in christian_300_600['lamma_sentence'] for word in sent.split())
fdist3 = FreqDist(word.lower() for sent in pagan_0_300['lamma_sentence'] for word in sent.split())
fdist4 = FreqDist(word.lower() for sent in pagan_300_600['lamma_sentence'] for word in sent.split())

# Find words that appear >=10 times in all subcorpora
vocab = set(
    word for word in fdist1 if fdist1[word] >= 10
) & set(
    word for word in fdist2 if fdist2[word] >= 10
) & set(
    word for word in fdist3 if fdist3[word] >= 10
) & set(
    word for word in fdist4 if fdist4[word] >= 10
)

In [4]:
import numpy as np
from collections import Counter
from itertools import combinations

def compute_pmi_from_sentences(sentences, vocab=vocab, min_count=10):
    # Tokenize sentences
    tokenized = [sent.split() for sent in sentences]
    # Flatten for unigram counts
    all_words = [w for sent in tokenized for w in sent]
    unigram_counts = Counter(all_words)
    # Filter vocab by min_count
    if vocab is None:
        vocab = set([w for w, c in unigram_counts.items() if c >= min_count])
    # Count co-occurrences (within sentence)
    cooc_counts = Counter()
    for sent in tokenized:
        words = [w for w in sent if w in vocab]
        for w1, w2 in combinations(sorted(set(words)), 2):
            cooc_counts[(w1, w2)] += 1
    N = len(tokenized)
    results = []
    for (w1, w2), c_xy in cooc_counts.items():
        c_x = unigram_counts[w1]
        c_y = unigram_counts[w2]
        # Probabilities
        p_x = c_x / N
        p_y = c_y / N
        p_xy = c_xy / N
        if p_xy > 0:
            pmi = np.log(p_xy / (p_x * p_y))
            results.append({'word1': w1, 'word2': w2, 'pmi': pmi, 'count': c_xy})
    return pd.DataFrame(results)

In [6]:
pmi_christian_0_300 = compute_pmi_from_sentences(christian_0_300['lamma_sentence'])
pmi_christian_300_600 = compute_pmi_from_sentences(christian_300_600['lamma_sentence'])
pmi_pagan_0_300 = compute_pmi_from_sentences(pagan_0_300['lamma_sentence'])
pmi_pagan_300_600 = compute_pmi_from_sentences(pagan_300_600['lamma_sentence'])

import pickle

pmi_dict = {
    "christian_0_300": pmi_christian_0_300,
    "christian_300_600": pmi_christian_300_600,
    "pagan_0_300": pmi_pagan_0_300,
    "pagan_300_600": pmi_pagan_300_600,
}

with open("../data/large-data/pmi_all_subcorpora.pkl", "wb") as f:
    pickle.dump(pmi_dict, f)

# text to label for zdenka
Zjeveni = lagt_tlg0031.tlg027
Matous = lagt_tlg0031.tlg001

In [ ]:
query = '''
SELECT t.sentence_id, w.*, s.text
FROM tokens t
JOIN works w ON t.grela_id = w.grela_id
JOIN sentences s ON t.sentence_id = s.sentence_id
WHERE w.lagt_provenience IN ('christian', 'pagan')
  AND t.lemma IN ('ἐχθρός')
  AND (
    (w.not_before > 0 AND w.not_before < 600)
    OR (w.not_after > 0 AND w.not_after < 600)
  )
'''
df = conn.execute(query).fetchdf()
df

,sentence_id,grela_source,grela_id,author,title,not_before,not_after,lagt_tlg_epithet,lagt_genre,lagt_provenience,...,noscemus_discipline,title_short,emlap_noscemus_id,place_publication,place_geonames,author_viaf,title_viaf,date_random,token_count,text
0,lagt_tlg0545.tlg003_92,lagt,lagt_tlg0545.tlg003,Aelian,ἐκ τῶν Αἰλιανοῦ ἀγροικικῶν ἐπιστολῶν,175.0,235.0,['Sophistae'],[],pagan,...,None,None,NaN,None,None,NaN,NaN,186.0,2455,τί γὰρ παθὼν ῥυθμίζεις με καὶ πρᾶον ἀποφῆναι γ...
1,lagt_tlg2200.tlg00427_48,lagt,lagt_tlg2200.tlg00427,Libanius,Oratio 27,385.0,385.0,['Rhetorici' 'Sophistae'],[],pagan,...,None,None,NaN,None,None,NaN,NaN,385.0,5267,οὐδέν οὖν ἕτερον λέγεις ἤ ὅτι φιλοῦσι τούς πον...
2,lagt_tlg0031.tlg014_29,lagt,lagt_tlg0031.tlg014,Pauline literature,New Testament - 2 Thessalonians,80.0,115.0,[],[],christian,...,None,None,NaN,None,None,NaN,NaN,85.0,941,"καὶ μὴ ὡς ἐχθρὸν ἡγεῖσθε, ἀλλὰ νουθετεῖτε ὡς ἀ..."
3,lagt_tlg0007.tlg070_4,lagt,lagt_tlg0007.tlg070,Plutarch,Πῶς ἄν τις διακρίνειε τὸν κόλακα τοῦ φίλου,96.0,120.0,['Biographi' 'Philosophici/-ae'],[],pagan,...,None,None,NaN,None,None,NaN,NaN,104.0,13729,"εἰ δὲ δὴ θεῖον ἡ ἀλήθεια καὶ "" πάντων μὲν ἀγαθ..."
4,lagt_tlg0007.tlg070_30,lagt,lagt_tlg0007.tlg070,Plutarch,Πῶς ἄν τις διακρίνειε τὸν κόλακα τοῦ φίλου,96.0,120.0,['Biographi' 'Philosophici/-ae'],[],pagan,...,None,None,NaN,None,None,NaN,NaN,104.0,13729,"ἡμεῖς δέ, εἰ μηδαμῆ μηδαμῶς ἐπαινοῦμεν τὸ "" ἐρ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3461,lagt_tlg0007.tlg027_134,lagt,lagt_tlg0007.tlg027,Plutarch,Philopoemen,68.0,120.0,['Biographi' 'Philosophici/-ae'],[],pagan,...,None,None,NaN,None,None,NaN,NaN,87.0,6386,"ἦν μὲν γὰρ ἐστεφανωμένους ἰδεῖν, ἦν δὲ τοὺς αὐ..."
3462,lagt_tlg0007.tlg035_60,lagt,lagt_tlg0007.tlg035,Plutarch,Κίμων,96.0,114.0,['Biographi' 'Philosophici/-ae'],[],pagan,...,None,None,NaN,None,None,NaN,NaN,101.0,6645,τὸν δ’ ὑπὸ τοῦ ψόφου ταραχθέντα καὶ σπασάμενον...
3463,lagt_tlg0007.tlg035_153,lagt,lagt_tlg0007.tlg035,Plutarch,Κίμων,96.0,114.0,['Biographi' 'Philosophici/-ae'],[],pagan,...,None,None,NaN,None,None,NaN,NaN,101.0,6645,ἐκεῖθεν δὲ ῥᾳδίως ἐπιβῆναι Μακεδονίας καὶ πολλ...
3464,lagt_tlg0007.tlg035_195,lagt,lagt_tlg0007.tlg035,Plutarch,Κίμων,96.0,114.0,['Biographi' 'Philosophici/-ae'],[],pagan,...,None,None,NaN,None,None,NaN,NaN,101.0,6645,ἡ δὲ βουλὴ τῶν πεντακοσίων πυθομένη καὶ φοβηθε...


In [6]:
# Sample 500 random christian and 500 random pagan rows
df_christian = df[df['lagt_provenience'] == 'christian'].sample(n=500, random_state=42)
df_pagan = df[df['lagt_provenience'] == 'pagan'].sample(n=500, random_state=42)

df_sampled = pd.concat([df_christian, df_pagan], ignore_index=True)
df_sampled.to_csv('../data/enemy_sample.csv')